In [1]:
from dataset import ProteinDataset

dataset = ProteinDataset('/gpfs/space/home/donatasv/semester_3/biological_data/project/data/preprocessed_small/biological_process_data.pkl')

/gpfs/space/home/donatasv/.conda/envs/bio/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# !git clone https://github.com/srush/annotated-s4

conda install jaxlib=*=*cuda* jax cuda-nvcc -c conda-forge -c nvidia

In [4]:
from annotated_s4.s4 import S4Layer, BatchStackedModel
import torch
import jax
from torch.utils.data import DataLoader, random_split
from functools import partial

In [5]:
from omegaconf import OmegaConf
config = {
    "model": {
        "d_model": 128,
        "n_layers": 4,
        "dropout": 0.0,
        "prenorm": True,
        "embedding": False,
        "layer": {
            "N": 64
        }
    },
    "train": {
        "epochs": 10,
        "bsz": 128,
        "lr": 0.001,
        "lr_schedule": False,
        "weight_decay": 0.01,
        "checkpoint": False,
        "suffix": None,  # Use None for null values in Python
        "sample": None
    }
}

config = OmegaConf.create(config)

In [6]:
seed = 1
torch.random.manual_seed(seed)  # For dataloader order
key = jax.random.PRNGKey(seed)
key, rng, train_rng = jax.random.split(key, num=3)

In [7]:
n_classes = len(dataset[0][1])
sequence_length = len(dataset[0][0])
input_dim = 1


total_size = len(dataset)
train_size = int(total_size * 0.9)
val_size = total_size - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [8]:
# Get model class and arguments
layer_cls = S4Layer
config['model']['layer']['l_max'] = sequence_length
# Extract custom hyperparameters from model class
lr_layer = getattr(layer_cls, "lr", None)
print(f"[*] Starting `{layer_cls}` Training on `{dataset}` =>> Initializing...")
model_cls = partial(
    BatchStackedModel,
    layer_cls=S4Layer,
    d_output=n_classes,
    classification=True,
    **config['model'],
)

[*] Starting `<class 'flax.linen.transforms.VmapS4Layer'>` Training on `<dataset.ProteinDataset object at 0x2ac973fa4a10>` =>> Initializing...


In [9]:
from annotated_s4.s4.train import create_train_state

2024-02-15 19:26:55.273894: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/blast-plus-2.12.0-zt4jna4xynkry22vk62pu2bzcspytxm7/lib:/gpfs/software/soft/spack/linux-centos7-broadwell/gcc-9.2.0/cuda-10.1.243-fgipoyn2aa7f5eqpjut35wchklelutf6/lib64:/gpfs/software/soft/spack/linux-centos7-broadwell/gcc-9.2.0/cudnn-7.6.3.30-10.1-linux-x64-2in3eqmrnltkm2jvegqk3momjizihy5t/lib64:/gpfs/space/software/cluster_software/compilers/gcc-9.2.0/lib64:/gpfs/space/software/cluster_software/manual/any/python/conda/3.8/lib:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/python-3.10.10-pfr7vmasg4bcyjnzs2fqbzu25nihahvn/lib:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/zlib-1.2.9-5f6os64uxcqi6maa562tiez2

In [10]:
# !pip install hydra optax flax==0.5.0 omegaconf

In [11]:
state = create_train_state(
    rng,
    model_cls,
    train_loader,
    lr=config.train.lr,
    lr_layer=lr_layer,
    lr_schedule=config.train.lr_schedule,
    weight_decay=config.train.weight_decay,
    total_steps=len(train_loader) * config.train.epochs,
)

/gpfs/space/home/donatasv/.conda/envs/bio/lib/python3.7/site-packages/flax/core/lift.py:112: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  scopes, treedef = jax.tree_flatten(scope_tree)
/gpfs/space/home/donatasv/.conda/envs/bio/lib/python3.7/site-packages/flax/core/lift.py:584: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  leaves = jax.tree_leaves(x)
/gpfs/space/home/donatasv/.conda/envs/bio/lib/python3.7/site-packages/flax/core/lift.py:592: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  axis_sizes = set(jax.tree_leaves(axis_sizes))


Traced<ShapedArray(float32[128])>with<BatchTrace(level=1/0)> with
  val = DeviceArray([[ 0.04555337,  0.01130313, -0.01651748, ...,  0.02575472,
               0.03717741, -0.0049449 ],
             [ 0.08009459, -0.0493723 , -0.01779799, ...,  0.00269891,
               0.01006696, -0.02034261],
             [ 0.04261799, -0.04334757,  0.01284971, ..., -0.04625276,
              -0.02381139, -0.04930048],
             ...,
             [ 0.05307975,  0.08503498,  0.00541616, ..., -0.06695689,
              -0.00237431,  0.01109306],
             [ 0.03436558,  0.01655401, -0.01819258, ...,  0.04164709,
               0.02646559, -0.05257869],
             [ 0.01717659,  0.04736052, -0.00554082, ...,  0.02223217,
               0.06620044, -0.06609014]], dtype=float32)
  batch_dim = 0


IndexError: tuple index out of range

In [ ]:
# # Loop over epochs
# best_loss, best_acc, best_epoch = 10000, 0, 0
# for epoch in range(train.epochs):
#     print(f"[*] Starting Training Epoch {epoch + 1}...")
#     state, train_loss, train_acc = train_epoch(
#         state,
#         train_rng,
#         model_cls,
#         trainloader,
#         classification=classification,
#     )
#     print(f"[*] Running Epoch {epoch + 1} Validation...")
#     test_loss, test_acc = validate(
#         state.params, model_cls, testloader, classification=classification
#     )
#     print(f"\n=>> Epoch {epoch + 1} Metrics ===")
#     print(
#         f"\tTrain Loss: {train_loss:.5f} -- Train Accuracy:"
#         f" {train_acc:.4f}\n\t Test Loss: {test_loss:.5f} --  Test"
#         f" Accuracy: {test_acc:.4f}"
#     )
#     # Save a checkpoint each epoch & handle best (test loss... not "copacetic" but ehh)
#     if train.checkpoint:
#         suf = f"-{train.suffix}" if train.suffix is not None else ""
#         run_id = f"checkpoints/{dataset}/{layer}-d_model={model.d_model}-lr={train.lr}-bsz={train.bsz}{suf}"
#         ckpt_path = checkpoints.save_checkpoint(
#             run_id,
#             state,
#             epoch,
#             keep=train.epochs,
#         )
#     if train.sample is not None:
#         if dataset == "mnist":  # Should work for QuickDraw too but untested
#             sample_fn = partial(
#                 sample_image_prefix, imshape=(28, 28)
#             )  # params=state["params"], length=784, bsz=64, prefix=train.sample)
#         else:
#             raise NotImplementedError(
#                 "Sampling currently only supported for MNIST"
#             )
#         # model_cls = partial(
#         #     BatchStackedModel,
#         #     layer_cls=layer_cls,
#         #     d_output=n_classes,
#         #     classification=classification,
#         #     **model,
#         # )
#         samples, examples = sample_fn(
#             # run_id,
#             params=state.params,
#             model=model_cls(decode=True, training=False),
#             rng=rng,
#             dataloader=testloader,
#             prefix=train.sample,
#             n_batches=1,
#             save=False,
#         )
#         if wandb is not None:
#             samples = [wandb.Image(sample) for sample in samples]
#             wandb.log({"samples": samples}, commit=False)
#             examples = [wandb.Image(example) for example in examples]
#             wandb.log({"examples": examples}, commit=False)
#     if (classification and test_acc > best_acc) or (
#         not classification and test_loss < best_loss
#     ):
#         # Create new "best-{step}.ckpt and remove old one
#         if train.checkpoint:
#             shutil.copy(ckpt_path, f"{run_id}/best_{epoch}")
#             if os.path.exists(f"{run_id}/best_{best_epoch}"):
#                 os.remove(f"{run_id}/best_{best_epoch}")
#         best_loss, best_acc, best_epoch = test_loss, test_acc, epoch
#     # Print best accuracy & loss so far...
#     print(
#         f"\tBest Test Loss: {best_loss:.5f} -- Best Test Accuracy:"
#         f" {best_acc:.4f} at Epoch {best_epoch + 1}\n"
#     )
#     if wandb is not None:
#         wandb.log(
#             {
#                 "train/loss": train_loss,
#                 "train/accuracy": train_acc,
#                 "test/loss": test_loss,
#                 "test/accuracy": test_acc,
#             },
#             step=epoch,
#         )
#         wandb.run.summary["Best Test Loss"] = best_loss
#         wandb.run.summary["Best Test Accuracy"] = best_acc
#         wandb.run.summary["Best Epoch"] = best_epoch